In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from env import api_key
import time

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

In [57]:
response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/battlebud?api_key={api_key}")

In [58]:
battlebud = response.json()

In [59]:
battlebud['puuid']

'8w5OLGrYy-YhUvX08ZD6a4sBvBvZuYTXU6UnfYhC1MBNo8rlV4OEODS9bNrqMBVuWirDBYOeMhWW0Q'

In [60]:
url = 'https://americas.api.riotgames.com/'

query = f'lol/match/v5/matches/by-puuid/8w5OLGrYy-YhUvX08ZD6a4sBvBvZuYTXU6UnfYhC1MBNo8rlV4OEODS9bNrqMBVuWirDBYOeMhWW0Q/ids?start=0&count=20&api_key={api_key}'

In [61]:
response = requests.get(url+query)

In [7]:
match_ids = response.json()

In [8]:
len(match_ids)

20

In [62]:
len(battlebud)

7

In [9]:
match_ids

['NA1_4083906028',
 'NA1_4083910411',
 'NA1_4083561526',
 'NA1_4069143161',
 'NA1_4069101224',
 'NA1_3967840881',
 'NA1_3967246118',
 'NA1_3966627103',
 'NA1_3966602533',
 'NA1_3966566502',
 'NA1_3966562962',
 'NA1_3966404211',
 'NA1_3966149816',
 'NA1_3965654430',
 'NA1_3965611323',
 'NA1_3965186467',
 'NA1_3965183441',
 'NA1_3964851235',
 'NA1_3964196435',
 'NA1_3963932851']

In [10]:
url = 'https://na.op.gg/ranking/ladder/'
response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
soup = BeautifulSoup(response.text)
table = soup.select('.ranking-table')
table = table[0]
table = soup.select('.ranking-table__cell')

In [11]:
table

[<td class="ranking-table__cell ranking-table__cell--rank">6</td>,
 <td class="ranking-table__cell ranking-table__cell--summoner"><a href="//na.op.gg/summoner/userName=Johnsun123"><img onerror="this.src='//opgg-static.akamaized.net/images/profile_icons/profileIcondefault.jpg';" src="//opgg-static.akamaized.net/images/profile_icons/profileIcon3785.jpg?image=c_scale,q_auto,c_scale,w_38&amp;v=1518361200"/><span>Johnsun123</span></a></td>,
 <td class="ranking-table__cell ranking-table__cell--tier">
 									Challenger
 								</td>,
 <td class="ranking-table__cell ranking-table__cell--lp">
 									1,517 LP
 								</td>,
 <td class="ranking-table__cell ranking-table__cell--level">
 								63
 							</td>,
 <td class="ranking-table__cell ranking-table__cell--winratio"><div class="winratio"><div class="winratio-graph"><div class="winratio-graph__fill winratio-graph__fill--left" style="width: 61%"></div><div class="winratio-graph__text winratio-graph__text--left">206</div><div class="w

In [47]:
def get_username(table):
    username_list = []
    for i in range (1,571,6):
        name = str(table[i])
        username = str(re.findall(r'userName=.+?"', name))
        username_list.append(username)
        df = pd.DataFrame(username_list)
        df = df.rename(columns={0:"pro"})
        df['pro'] = df.pro.str.replace("\['userName=","")
        df['pro'] = df.pro.str.replace("+",' ')
        df['pro'] = df.pro.str.replace("\"']",'')
        df = df[~df.pro.str.contains('%')]
        df = df.reset_index()
    return df

In [52]:
def op_gg_webscrape(pages):
    player_names = []
    for i in range(pages + 1):
        url = (f'https://na.op.gg/ranking/ladder/page={i}')
        response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
        soup = BeautifulSoup(response.text)
        table = soup.select('.ranking-table')
        table = table[0]
        table = soup.select('.ranking-table__cell')
        users_df = get_username(table)
        for pro in users_df.pro:
                player_names.append(str(pro))
    return player_names

In [53]:
player_list = op_gg_webscrape(30)

<ipython-input-47-c953d951acde>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['pro'] = df.pro.str.replace("\['userName=","")
<ipython-input-47-c953d951acde>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['pro'] = df.pro.str.replace("+",' ')
<ipython-input-47-c953d951acde>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['pro'] = df.pro.str.replace("\"']",'')


In [54]:
player_list = list(set(player_list))

In [55]:
len(player_list)

2641

In [63]:
player_df = pd.DataFrame({'players':player_list})

In [66]:
player_df.players

0                  Ahwii
1                 Blazze
2              Prabhably
3        Dark Rose Vayne
4               hyperton
              ...       
2636            Denathor
2637              Impmon
2638    personnel office
2639          EBIC GAMER
2640     Muslim Salesman
Name: players, Length: 2641, dtype: object

In [18]:
player_df.to_csv('2641_players.csv')

In [19]:
def op_gg_webscrape(pages):
    player_names = []
    url = 'https://na.op.gg/ranking/ladder/'
    response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
    soup = BeautifulSoup(response.text)
    table = soup.select('.ranking-table')
    table = table[0]
    table = soup.select('.ranking-table__cell')
    users_df = get_username(table)
    for pro in users_df.pro:
        player_names.append(pro)
        for i in range(pages + 1):
            url = (f'https://na.op.gg/ranking/ladder/page={i}')
            response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
            soup = BeautifulSoup(response.text)
            table = soup.select('.ranking-table')
            table = table[0]
            table = soup.select('.ranking-table__cell')
            users_df = get_username(table)
            for pro in users_df.pro:
                player_names.append(pro)
    return player_names

In [ ]:
player_list

In [ ]:
player_names = []
url = 'https://na.op.gg/ranking/ladder/'
response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
soup = BeautifulSoup(response.text)
table = soup.select('.ranking-table')
table = table[0]
names = soup.select('.ranking-table__cell')
for i in range (1,571,6):
    #summoner_names = re.match(r'userName=.*',str(names[i]))
    name = str(names[i])
    username = str(re.findall(r'userName=.+?"', name))
    player_names.append(username)

In [ ]:
Tçº

In [ ]:
df = pd.DataFrame(player_names)

In [ ]:
df = df.rename(columns={0:"pro"})

In [ ]:
df

In [ ]:
df['pro'] = df.pro.str.replace("\['userName=","")

In [ ]:
df['pro'] = df.pro.str.replace("+",' ')

In [ ]:
df['pro'] = df.pro.str.replace("\"']",'')

In [ ]:
df.shape

In [ ]:
df['pro'] = df.where(df.pro.str.contains('%') == False)

In [ ]:
df = df[~df.pro.str.contains('%')]

In [ ]:
df

In [ ]:
df.sample(10)

In [28]:
df = pd.read_csv('more_players.csv')

In [15]:
df = pd.DataFrame(columns = ['puuid','username'])

In [21]:
puuid = {'puuid':'ddsdafdsgf'}
puuid2 = {'puuid':'ddsdafdsasdadgf'}

In [25]:
df = df.append(puuid,ignore_index = True)

In [26]:
df.append(puuid2,ignore_index = True)

,puuid,username
0,ddsdafdsgf,NaN
1,ddsdafdsasdadgf,NaN


In [56]:
len(battlebud)

NameError: name 'battlebud' is not defined

In [29]:
battlebud['puuid']

'8w5OLGrYy-YhUvX08ZD6a4sBvBvZuYTXU6UnfYhC1MBNo8rlV4OEODS9bNrqMBVuWirDBYOeMhWW0Q'

In [32]:
battlebud['name']

'Battlebud'

In [30]:
info = {'puuid':battlebud['puuid']}

In [31]:
info

{'puuid': '8w5OLGrYy-YhUvX08ZD6a4sBvBvZuYTXU6UnfYhC1MBNo8rlV4OEODS9bNrqMBVuWirDBYOeMhWW0Q'}

In [39]:
name_list = df.players.head(40)

In [67]:
def get_puuid(name_list,df):
    counter = 0
    for name in name_list:
        print(counter)
        counter += 1
        if counter%50 == 0:
            print('function paused')
            print
            time.sleep(180)
            print('Continue')
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            if len(summoner_info) == 7:
                info = {'puuid':summoner_info['puuid'],
                       'username':summoner_info['name']}
                df = df.append(info,ignore_index = True)
            else:
                print(summoner_info)
        else:
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            if len(summoner_info) == 7:
                info = {'puuid':summoner_info['puuid'],
                       'username':summoner_info['name']}
                df = df.append(info,ignore_index = True)
            else:
                print(summoner_info)
    return df

In [68]:
username_df = pd.DataFrame(columns = ['puuid','username'])

In [ ]:
df = get_puuid(player_df.players,username_df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
function paused
Continue
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
function paused


In [44]:
df

,puuid,username
0,Mw35Ro1ToceTVVq2snwUHxWTpLOi1_gxX-dwdA_wzdryOh...,Satoshi
1,ryVelN921c-1K6GuI4UB-Rd_lNF3Ovw8jokEvPVRT5rSvK...,Johnsun123
2,YSblrqqkqRSduZds0m-BkkZzvzn8_DdzLq0-bPUj6r3OxE...,bogo binter
3,oCBp_DRp8IFqM1-jnfNua_ovEdDUtEnMJS2Hi4oSvfxYm4...,Sybr
4,e6IRN3YnxoP-YN-wI7MQYSxsLcLzcpFq0UFoKC-2pt0jP6...,good nightt
5,8OsDeXKP_fQuEqskUuuZkwFOOU32ETJDJt9e7SLUevWmzJ...,Lost Kin
6,KsJfBZh6ICMznOPnWvl6lhxc6c4Awyuwd-sKrdEnjDjbi5...,DIG Spawn
7,1Wpnf74hTJNTQiNpp6JSEogspsgoL1UwcLBj435_hUUCQK...,Leemo
8,tquL2JUzSRSKBziNNFglbyFIPI80C2sI8IXn7S6BeXO2dp...,LeenK LCS
9,l-Giu5FtnEC5OzT43i-DLkqu5q6ZKCwiQCx-P61iW0d8vW...,Diomarr


In [17]:
def get_puuid(name_list):
    puuid_list = []
    counter = 0
    for name in name_list:
        print(counter)
        counter += 1
        if counter%50 == 0:
            print('function paused')
            print
            time.sleep(180)
            print(puuid_list[counter - 5])
            print('Continue')
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info['puuid'])
        else:
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info['puuid'])
    return puuid_list

In [16]:
puuid_list

NameError: name 'puuid_list' is not defined

In [ ]:
response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/battlebud?api_key={api_key}")

In [ ]:
summoner_info = response.json()

In [ ]:
battlebud